At a high level, decorators are a way to modify or enhance existing functions in a non-intrusive and maintainable way.

In Python, a decorator is a callable object that takes in a callable and returns a callable.  Decorators can be generally be thought of as functions that take a function as an argument and return another function.

In the example below, we can use the built-in ascii() function to convert all non-ASCII character to escape sequences:

In [2]:
def escape_unicode(f):
    def wrap(*args, **kwargs):
        x = f(*args, **kwargs)
        return ascii(x)

    return wrap

In the above, the decorator, escape_unicode, is just a normal function.  Its only argument f, is the function to be decorated.  

It is important to notice that escape_unicode returns wrap.  Remember that  a decorator takes a a callable as its argument and returns a new callable.  In this case, the new callable is wrap.  By using closures, wrap is able to use the parameter f even after escape_unicode has returned.

Now that we have a decorator, create a function that will benefit from it:

In [3]:
def northern_city():
    return 'Tromsø'

In [4]:
print(northern_city())

Tromsø


To add unicode escaping to the function, simply decorate northern_city with the escape_unicode decorator:

In [8]:
@escape_unicode
def northern_city():
    return 'Tromsø'

Now when calling northern_city non-ASCII characters are converted to escape sequences

In [9]:
print(northern_city())

'Troms\xf8'


Second Example:

Because wrapper() is a regular Python function, the way a decorator modifies a function can change dynamically. So as not to disturb your neighbors, the following example will only run the decorated code during the day:

In [14]:
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            pass  # Hush, the neighbors are asleep
    return wrapper

def say_whee():
    print("Whee!")

say_whee = not_during_the_night(say_whee)

If you try to call say_whee() after bedtime, nothing will happen:

In [15]:
say_whee()

Whee!


Decorating Functions With Arguments

Say that you have a function that accepts some arguments. Can you still decorate it? Let’s try:

In [16]:
from decorators import do_twice

@do_twice
def greet(name):
    print(f"Hello {name}")

Unfortunately, running this code raises an error:

In [17]:
greet("World")

TypeError: wrapper_do_twice() takes 0 positional arguments but 1 was given

The problem is that the inner function wrapper_do_twice() does not take any arguments, but name="World" was passed to it. You could fix this by letting wrapper_do_twice() accept one argument, but then it would not work for the say_whee() function you created earlier.

The solution is to use *args and **kwargs in the inner wrapper function. Then it will accept an arbitrary number of positional and keyword arguments. Rewrite decorators.py as follows:

In [18]:
from modified_decorators import do_twice

@do_twice
def greet(name):
    print(f"Hello {name}")

In [19]:
greet("World")

Hello World
Hello World


Returning Values From Decorated Functions:

What happens to the return value of decorated functions? Well, that’s up to the decorator to decide. Let’s say you decorate a simple function as follows:

In [20]:
from modified_decorators import do_twice

@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

Try to use it:

In [21]:
hi_adam = return_greeting("Adam")

Creating greeting
Creating greeting


Oops, your decorator ate the return value from the function.

Because the do_twice_wrapper() doesn’t explicitly return a value, the call return_greeting("Adam") ended up returning None.

To fix this, you need to make sure the wrapper function returns the return value of the decorated function. Change your decorators.py file:

In [22]:
from second_modified_decorators import do_twice

@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

The return value from the last execution of the function is returned:

In [23]:
return_greeting("Adam")

Creating greeting
Creating greeting


'Hi Adam'